In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["MKL_NUM_THREADS"] = "8" 
os.environ["NUMEXPR_NUM_THREADS"] = "8" 
os.environ["OMP_NUM_THREADS"] = "8" 

In [2]:
import rnacomp
import rnacomp.models
from rnacomp.fastai_fit import *
from rnacomp.dataset import LenMatchBatchSampler, DeviceDataLoader
from rnacomp.utils import seed_everything, MAE, loss_laplace
import gc
import wandb 
from fastxtend.vision.all import EMACallback


In [3]:
!ls ../data/

Ribonanza_bpp_files	  split
all_test_data.npy	  supplementary_silico_predictions
eterna_openknot_metadata  test_sequences.csv
rcentral.parquet	  train_corrected.parquet
rmdb_data.v1.3.0.csv	  train_data.csv
sample_submission.csv	  train_data.parquet
sequence_libraries


In [4]:
class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    split_id = Path('../eda/fold_split.csv')
    bs = 64
    num_workers = 4
    device = 'cuda'
    seed = 2023
    out = 'exp_32_v3'
    dataset_name = 'RNA_DatasetBaselineSplitssbppV6SAVEDwithFM'
    sn_train = False
    
    model_name = 'RNA_ModelV25'
    model_kwargs = dict(dim=192 * 2,
        depth=4,
        head_size=32,
        drop_pat_dropout=0.2,
        dropout=0.2,
        bpp_transfomer_depth = 6)

   
    epoch = 64
    lr = 5e-4
    wd = 0.05
    pct_start = 0.02
    
    wandb = True
    
seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

def class_to_dict(cls):
    # Create a dictionary from the class attributes
    return {key: value for key, value in cls.__dict__.items() if not key.startswith("__") and not callable(value)}

In [5]:



split = pd.read_csv(CFG.split_id)
df = pd.read_parquet(CFG.path/'train_corrected.parquet')
df = pd.merge(df, split, on='sequence_id')
#df = df.query("SN_filter==1").reset_index(drop=True)
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)


ds_train = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train',sn_train=CFG.sn_train)
ds_train_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_train, mode='train', mask_only=True,sn_train=CFG.sn_train)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval')
ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

data = DataLoaders(dl_train,dl_val)
del split
del df
gc.collect()



62

In [ ]:
#the training was interepeted on epoch 3 
#so i had to restart from scratch but loading weights from epoch 3
learn = Learner(data,
                getattr(rnacomp.models, CFG.model_name)(**CFG.model_kwargs).cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                    #WandbCallback(log_preds=False),
                    CSVLogger(),
                     EMACallback(replace_weights=True),
                    SaveModelCallback(monitor='mae',comp=np.less,at_end=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(CFG.epoch, lr_max=CFG.lr, wd=CFG.wd, pct_start=CFG.pct_start)
wandb.finish()

epoch,train_loss,valid_loss,mae,time
0,0.122348,0.203911,0.171251,55:46
1,0.117241,0.143638,0.143221,55:45
2,0.114877,0.124242,0.135304,55:52
3,0.114367,0.122112,0.133024,55:52
4,0.112835,0.120868,0.131772,55:33
5,0.113828,0.120165,0.131074,55:30
6,0.113295,0.119722,0.130617,55:28
7,0.112380,0.119354,0.130227,55:29
8,0.111752,0.118977,0.129816,55:34
9,0.110586,0.118651,0.129468,55:34


Better model found at epoch 0 with mae value: 0.17125069591886063.
Better model found at epoch 1 with mae value: 0.1432211487851233.
Better model found at epoch 2 with mae value: 0.13530391835629355.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Better model found at epoch 13 with mae value: 0.12837749013292457.
Better model found at epoch 14 with mae value: 0.12822905154903394.
Better model found at epoch 15 with mae value: 0.12804676082320887.
Better model found at epoch 16 with mae value: 0.127835245679542.
Better model found at epoch 17 with mae value: 0.12771217845509797.
Better model found at epoch 18 with mae value: 0.12744226934076133.
Better model found at epoch 19 with mae value: 0.12732011912673194.
Better model found at epoch 20 with mae value: 0.12718340550822577.
Better model found at epoch 21 with mae value: 0.12704308221453006.
Better model found at epoch 22 with mae value: 0.12695075208158446.
Better model found at epoch 23 with mae value: 0.12690513114465354.
Better model found at epoch 24 with mae value: 0.12671634057642384.
Better model found at epoch 25 with mae value: 0.12667348844061796.
Better model found at epoch 26 with mae value: 0.12659252386577496.
Better model found at epoch 27 with mae value: 0.1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
res = dict()
for l in df_valid["L"].unique():
    ds_val = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval')
    ds_val_len = getattr(rnacomp.dataset, CFG.dataset_name)(df_valid.query("L==@l").copy(), mode='eval', mask_only=True)
    sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
    len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
                   drop_last=False)
    dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
                   batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

    data = DataLoaders(dl_val,dl_val)
    learn = Learner(data,
                    getattr(rnacomp.models, CFG.model_name)(dim = CFG.dim, graph_layers=CFG.graph_layers).cuda(), 
                    path = CFG.out, 
                    loss_func=loss_laplace,
                    metrics=[MAE()]).to_fp16() 
    learn.load('model')
    learn.eval()
    loss_, score_ = learn.validate()
    res[l]= score_

res = pd.DataFrame(pd.Series(res)).reset_index()
res.columns = ['L', 'mae']
res.sort_values(by="L")

In [ ]:
res

In [ ]:
res